In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# %load_ext line_profiler

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Setup the models that we will test

In [ ]:
from lotka_volterra_model import LotkaVolterraModel
from dimerization_model import DimerizationModel
from gene_toggle_model import GeneToggleModel
from fast_slow_model import FastSlowModel
from goutsias_model import GoutsiasModel

from model_data import ModelData
        
all_model_data = []
initial_distributions = []
max_r_for_all_models = []
r_steps = []

# ########################
# # Lokta-Volterra model
# ########################
all_model_data.append(ModelData(LotkaVolterraModel(),
                                ['A', 'B'],
                                0.2,
                                'Lotka-Volterra'))
initial_distributions.append({(300, 100): 1.})
max_r_for_all_models.append(250)
r_steps.append(10)

# ########################
# # Dimerization model
# ########################
all_model_data.append(ModelData(DimerizationModel(),
                                ['mRNA', 'protein', 'dimer'],
                                0.05,
                                'Dimerization'))
initial_distributions.append({(10, 10, 0): 1.})
max_r_for_all_models.append(150)
r_steps.append(10)

########################
# Fast/slow model
########################
all_model_data.append(ModelData(FastSlowModel(),
                                ['A', 'B', 'C'],
                                4.,
                                'Fast/Slow'))
initial_distributions.append({(100, 100, 0): 1.})
max_r_for_all_models.append(150)
r_steps.append(10)

# #######################
# Goutsias model
# #######################
all_model_data.append(ModelData(GoutsiasModel(),
                                ['M', 'DNA', 'RNA', 'D', 'DNA.D', 'DNA.2D'],
                                0.2,
                                'Goutsias'))
initial_distributions.append({(20, 20, 20, 20, 20, 20): 1.})
max_r_for_all_models.append(20)
r_steps.append(1)

In [ ]:
# from run_sim import run_sim
# state_path, _ = run_sim(all_model_data[0].model, np.array([10, 10, 10, 10, 10, 10]), 1.)
# print(state_path[-1,1:])

# Generate the plots comparing the support sizes to the probability lost

In [ ]:
import r_step_reachability
import branch_and_prune
import gorde

from get_generator import get_generator
from scipy.sparse.linalg import expm_multiply

for model_data, init_dist, max_r, r_step in zip(all_model_data, initial_distributions, max_r_for_all_models, r_steps):
    print(model_data.model_name)
    r_values = range(1, max_r, r_step)
    r_step_support_size = np.zeros(len(r_values), dtype=np.int)
    r_step_prob_lost = np.zeros(len(r_values), dtype=np.float)
    
    ############
    # r-step
    ############
    for i,r in enumerate(r_values):
        print(r)
        support = r_step_reachability.get_support(model_data, init_dist, r)
        generator, initial_dist, index_to_state = get_generator(model_data, init_dist, support)
        res_dist = expm_multiply(generator * model_data.t, initial_dist)
        
        r_step_support_size[i] = len(support)
        r_step_prob_lost[i] = res_dist[0]
        print('support size: {}, prob lost: {}'.format(len(support), res_dist[0]))
    
    ############
    # branch--and--prune
    ############
    branch_prune_support_size = []
    branch_prune_prob_lost = []
    
    eps = 1.
    prob_lost = np.inf
    print('error target: {:.2e}'.format(r_step_prob_lost[-1]))
    while prob_lost > r_step_prob_lost[-1]:
        support, generator, initial_dist = branch_and_prune.get_support(model_data, init_dist, eps)
        res_dist = expm_multiply(generator.T * model_data.t, initial_dist)
        branch_prune_support_size.append(len(support))
        branch_prune_prob_lost.append(res_dist[0])
        
        eps /= 4.
        prob_lost = res_dist[0]
        print('eps: {:.2e}, prob lost: {:.2e}, support size: {}'.format(eps, prob_lost, len(support)))
        
    plt.plot(r_step_support_size, r_step_prob_lost, color='blue', label='r-step reachability')
    plt.plot(branch_prune_support_size, branch_prune_prob_lost, color='red', label='branch and prune')
    plt.legend()
    plt.yscale('log') #base 10
    plt.ylabel('L1 error')
    plt.xlabel('Domain size')
    plt.title(model_data.model_name)
    
    plt.gcf().set_size_inches(8, 6)
    plt.savefig('{}_domain_scaling.eps'.format(model_data.model_name.replace('/', '')),
            format='eps',
            bbox_inches='tight')
    
    plt.show()

# test L1 error

In [ ]:
def compute_l1_diff(support_1, initial_dist_array_1, support_2, initial_dist_array_2):
    for state,index in support_1.items():
        print(state,index)

# model_data = ModelData(LotkaVolterraModel(),
#                        ['A', 'B'],
#                        0.1,
#                        'Lotka-Volterra')
# init_dist = {(300, 100): 1.}

model_data = ModelData(DimerizationModel(),
                       ['mRNA', 'protein', 'dimer'],
                       0.1,
                       'Dimerization')
init_dist = {(10, 10, 0): 1.}

support = r_step_reachability.get_support(model_data, init_dist, 200)
generator, initial_dist, index_to_state = get_generator(model_data, init_dist, support)
high_res_dist = expm_multiply(generator * model_data.t, initial_dist)

bp_support, generator, initial_dist = branch_and_prune.get_support(model_data, init_dist, 10**(-3))
bp_dist = expm_multiply(generator.T * model_data.t, initial_dist)

l1_error = 0
for reach_index, state in index_to_state.items():
    if reach_index == 0:
        continue
        
    if state in bp_support:
        bp_index = bp_support[state]
        l1_error += abs(bp_dist[bp_index] - high_res_dist[reach_index])
    else:
        l1_error += high_res_dist[reach_index]
        
print('L1 error: {}'.format(l1_error))
print('high res sink: {}'.format(high_res_dist[0]))
print('BP sink: {}'.format(bp_dist[0]))